In [ ]:
# 必要なモジュールのimport
import numpy as np
import pandas as pd
pd.set_option('max_columns', 150) # pandas dataframe表示列数の設定
import gc

import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.patches as patches

import seaborn as sns
from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.express as px
import plotly.graph_objs as go
import datetime as dt

import os, random, math, psutil, pickle

# kaggle kernelの利用を想定しています。
# ローカルなど、別環境で作業する場合はファイルを置いたディレクトリを指定してください
print(os.listdir('../input/ashrae-energy-prediction/')) 

In [ ]:
%%time
# %%timeはセルの実行時間を計測

# データの読み込み
train_df = pd.read_csv('../input/ashrae-energy-prediction/train.csv')
test_df = pd.read_csv('../input/ashrae-energy-prediction/test.csv')
weather_train_df = pd.read_csv('../input/ashrae-energy-prediction/weather_train.csv')
weather_test_df = pd.read_csv('../input/ashrae-energy-prediction/weather_test.csv')
building_meta_df = pd.read_csv('../input/ashrae-energy-prediction/building_metadata.csv')
sample_submission = pd.read_csv('../input/ashrae-energy-prediction/sample_submission.csv')

# 文字列をTimestamp型に変換（日時情報の処理をしやすくするためです）
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
test_df['timestamp'] = pd.to_datetime(test_df['timestamp'])
weather_train_df['timestamp'] = pd.to_datetime(weather_train_df['timestamp'])
weather_test_df['timestamp'] = pd.to_datetime(weather_test_df['timestamp'])

In [ ]:
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

def reduce_mem_usage(df, use_float16=False):
    start_mem = df.memory_usage().sum()/1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            print(col)
            continue
        col_type = df[col].dtype
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
%%time
train_df = reduce_mem_usage(train_df, use_float16=True)
building_meta_df = reduce_mem_usage(building_meta_df, use_float16=True)
weather_train_df = reduce_mem_usage(weather_train_df, use_float16=True)
weather_test_df = reduce_mem_usage(weather_test_df, use_float16=True)
sample_submission = reduce_mem_usage(sample_submission, use_float16=True)

In [ ]:
# test_dfもほぼ同じ形状(meter_readingカラムがないだけ)なので省略
train_df.head()

In [ ]:
building_meta_df.head()

In [ ]:
# weather_test_dfは同じ形状
weather_train_df.head()

In [ ]:
# dataframe形状の確認
print('train_df 形状: ', train_df.shape)
print('test_df 形状: ', test_df.shape)
print('building_meta_df 形状: ', building_meta_df.shape)
print('weather_train_df 形状: ', weather_train_df.shape)
print('weather_test_df 形状: ', weather_test_df.shape)

In [ ]:
# 欠損値（の割合）確認
train_df.count()/len(train_df)

In [ ]:
test_df.count()/len(test_df)

In [ ]:
# year_built/floor_countに欠損が見られる
building_meta_df.count()/len(building_meta_df)

In [ ]:
# train/testどちらのデータでもsite_id/timestamp以外のカラムに欠損が見られます
weather_train_df.count()/len(weather_train_df)

In [ ]:
weather_test_df.count()/len(weather_test_df)

In [ ]:
# dataframeの結合
train_df = train_df.merge(building_meta_df, on='building_id', how='left')
test_df = test_df.merge(building_meta_df, on='building_id', how='left')

train_df = train_df.merge(weather_train_df, on=['site_id', 'timestamp'], how='left')
test_df = test_df.merge(weather_test_df, on=['site_id', 'timestamp'], how='left')

# delで削除して、gc.collect()ですぐにメモリを解放する
del weather_train_df, weather_test_df, building_meta_df
gc.collect();

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(12, 4), dpi=100)
# 1時間毎の平均の消費量について可視化
train_df[['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes, label='By hour', alpha=0.8).set_ylabel('Meter reading', fontsize=14);
# 1日毎の平均の消費量について可視化
train_df[['timestamp', 'meter_reading']].set_index('timestamp').resample('D').mean()['meter_reading'].plot(ax=axes, label='By day', alpha=1).set_ylabel('Meter reading', fontsize=14);

axes.set_title('Mean Meter reading by hour and day', fontsize=16);
axes.legend();

In [ ]:
# siteごとに可視化する。15siteあるので8x2で表示するようにします。
fig, axes = plt.subplots(8, 2, figsize=(12, 28), dpi=100)
# site_idの値でループ
for i in range(train_df['site_id'].nunique()):
    # site_id = iのデータを取得
    train_df[train_df['site_id'] == i][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[i%8][i//8], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13);
    train_df[train_df['site_id'] == i][['timestamp', 'meter_reading']].set_index('timestamp').resample('D').mean()['meter_reading'].plot(ax=axes[i%8][i//8], alpha=1, label='By day', color='tab:orange').set_xlabel('');
    axes[i%8][i//8].legend();
    axes[i%8][i//8].set_title('site_id {}'.format(i), fontsize=13);
    plt.subplots_adjust(hspace=0.45)

In [ ]:
fig, axes = plt.subplots(8, 2, figsize=(10, 18), dpi=100)
# value_counts()のindexをリスト化
for i, use in enumerate(train_df['primary_use'].value_counts().index.to_list()):
    try:
        train_df[(train_df['site_id']==13) & (train_df['primary_use'] == use)][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[i%8][i//8], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=7);
        train_df[(train_df['site_id']==13) & (train_df['primary_use'] == use)][['timestamp', 'meter_reading']].set_index('timestamp').resample('D').mean()['meter_reading'].plot(ax=axes[i%8][i//8], alpha=1, label='By day', color='tab:orange').set_xlabel();
        axes[i%8][i//8].legend();
    except TypeError:
        pass
    axes[i%8][i//8].set_title(use, fontsize=7);
    plt.subplots_adjust(hspace=0.8)

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(5, 7), dpi=100)
for i in train_df[(train_df['site_id'] == 13) & (train_df['primary_use'] == 'Education') ]['meter'].value_counts(dropna=False).index.to_list():
    train_df[(train_df['site_id'] == 13) & (train_df['primary_use'] == 'Education') & (train_df['meter'] == i)][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[i], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=6);
    train_df[(train_df['site_id'] == 13) & (train_df['primary_use'] == 'Education') & (train_df['meter'] == i)][['timestamp', 'meter_reading']].set_index('timestamp').resample('D').mean()['meter_reading'].plot(ax=axes[i], alpha=1, label='By day', color='tab:orange').set_xlabel('');
    axes[i].legend();
    axes[i].set_title('Meter: '+ str(i), fontsize=5);
    plt.subplots_adjust(hspace=0.6)

In [ ]:
fig, axes = plt.subplots(9, 2, figsize=(10, 20), dpi=100)
for i, build in enumerate(train_df[(train_df['site_id'] == 13) & (train_df['primary_use'] == 'Education') & (train_df['meter'] == 2)]['building_id'].value_counts(dropna=False).index.to_list()):
    train_df[(train_df['site_id'] == 13) & (train_df['primary_use'] == 'Education') & (train_df['meter'] == 2) & (train_df['building_id'] == build)][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[i%9][i//9], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=6);
    train_df[(train_df['site_id'] == 13) & (train_df['primary_use'] == 'Education') & (train_df['meter'] == 2) & (train_df['building_id'] == build)][['timestamp', 'meter_reading']].set_index('timestamp').resample('D').mean()['meter_reading'].plot(ax=axes[i%9][i//9], alpha=1, label='By day', color='tab:orange').set_xlabel('');
    axes[i%9][i//9].legend();
    axes[i%9][i//9].set_title('Meter: '+ str(build), fontsize=5);
    plt.subplots_adjust(hspace=0.6)

In [ ]:
def data_cleanup(df, interval=24):
    df["datetime"] = pd.to_datetime(df["timestamp"], format="%Y-%m-%d %H:%M:%S")
    df['month'] = df.datetime.dt.month      
    df['day'] = df.datetime.dt.day      
    df['hour'] = df.datetime.dt.hour       
    # 異常データの削除(Discussionで情報共有)      
    df = df[df['building_id'] != 1099]      
    df = df[~((df.building_id<=104) & (df.meter==0) & ((df.month<=5) & (df.day<=20)))]                   

    # meter_reading=0のデータをmeterごとにbooleanで持つ
    is_electric = (df.meter==0) & (df.meter_reading==0)
    is_chilledwater = (df.meter==1) & (df.meter_reading==0)
    is_steam = (df.meter==2) & (df.meter_reading==0)
    is_hotwater = (df.meter==3) & (df.meter_reading==0)

    # electric(電気)の消費量が0になるのは、時期に限らずおかしい
    bad_electric = df[is_electric].index
    df = df.drop(bad_electric, axis=0)
    del is_electric, bad_electric
    gc.collect()

    # shilledwater（冷水）は冬の間は0になっていても問題ないと考える
    chilled_transitions = is_chilledwater != is_chilledwater.shift(1)
    # 前後のデータと同じかどうか
    chilled_sequence = chilled_transitions.cumsum()
    # 累積和を計算
    chilled_ids = chilled_sequence[is_chilledwater].rename('ids')
    # 前後連続して0かどうかの情報取得
    keep = chilled_ids[(df.datetime>= dt.datetime(2016,1,15)) & (df.datetime <= dt.datetime(2016,12,15))].unique()
    # 冬の期間は0でも問題ないので、それ以外を指定する
    is_bad = chilled_ids.isin(keep) & (chilled_ids.map(chilled_ids.value_counts() >= 24))
    result = is_chilledwater.copy()
    result.update(is_bad)
    chilled_index = df[result].index
    df = df.drop(chilled_index, axis=0)
    del is_chilledwater, chilled_transitions, chilled_sequence, chilled_ids, keep, is_bad, result, chilled_index
    gc.collect()

    # steam(蒸気)、hotwater(温水)は夏の間0でも問題ないと考える
    steam_transitions = is_steam != is_steam.shift(1)
    steam_sequence = steam_transitions.cumsum()
    steam_ids = steam_sequence[is_steam].rename('ids')
    keep = steam_ids[(df.datetime <= dt.datetime(2016,8,1)) | (df.datetime >= dt.datetime(2016,8,31))].unique()
    is_bad = steam_ids.isin(keep) & (steam_ids.map(steam_ids.value_counts()>=24))
    result = is_steam.copy() 
    result.update(is_bad)
    steam_index = df[result].index
    df = df.drop(steam_index, axis=0)
    del is_steam, steam_transitions, steam_sequence, steam_ids, keep, is_bad, result, steam_index
    gc.collect()

    hotwater_transitions = is_hotwater != is_hotwater.shift(1)
    hotwater_sequence = hotwater_transitions.cumsum()
    hotwater_ids = hotwater_sequence[is_hotwater].rename('ids')
    keep = hotwater_ids[(df.datetime <= dt.datetime(2016,8,1)) | (df.datetime >= dt.datetime(2016,8,31))].unique()
    is_bad = hotwater_ids.isin(keep) & (hotwater_ids.map(hotwater_ids.value_counts()>=24))
    result = is_hotwater.copy()
    result.update(is_bad)
    hotwater_index = df[result].index
    df = df.drop(hotwater_index, axis=0)
    df = df.drop(['month', 'day', 'hour', 'datetime'], axis=1)
    del is_hotwater, hotwater_transitions, hotwater_sequence, hotwater_ids, keep, is_bad, result, hotwater_index
    gc.collect()

    return df

In [ ]:
# trainデータでの'meter'の出現頻度を抽出
train_data = train_df['meter'].value_counts(dropna=False, normalize=True).sort_index().values
ind = np.arange(len(train_data))
width = 0.35

fig, axes = plt.subplots(1, 1, figsize=(14, 6), dpi=100)
tr = axes.bar(ind, train_data, width, color='royalblue')
# testデータでの'meter'の出現頻度を抽出
test_data = test_df['meter'].value_counts(dropna=False, normalize=True).sort_index().values
tt = axes.bar(ind+width, test_data, width, color='seagreen')

axes.set_ylabel('Normalized number of observations');
axes.set_xlabel('meter type');
axes.set_xticks(ind + width / 2)

axes.set_xticklabels(train_df['meter'].value_counts().sort_index().index, rotation=0)
axes2 = axes.twinx()

# 'meter'でグループ化して、'meter_reading'の平均を計算
mr = axes2.plot(ind, train_df[['meter', 'meter_reading']].groupby('meter')['meter_reading'].mean().sort_index().values, 'D-', color='tab:orange', label='Mean meter reading');
axes2.grid(False);
axes2.tick_params(axis='y', labelcolor='tab:orange');
axes2.set_ylabel('Mean meter reading by meter type', color='tab:orange');
axes.legend([tr, tt], ['Train', 'Test'], facecolor='white');
axes2.legend(loc=5, facecolor='white');

In [ ]:
# site(地域)ごとではなく、全体として確認する
fig, axes = plt.subplots(1,1,figsize=(14, 6), dpi=100)
# train_dfから'timestamp'と'air_temperature'カラムを取得し、'timestamp'
train_df[['timestamp', 'air_temperature']].set_index('timestamp').resample('H').mean()['air_temperature'].plot(ax=axes, alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean temperature', fontsize=14);
test_df[['timestamp', 'air_temperature']].set_index('timestamp').resample('H').mean()['air_temperature'].plot(ax=axes, alpha=0.8, color='tab:blue', label='');
train_df[['timestamp', 'air_temperature']].set_index('timestamp').resample('D').mean()['air_temperature'].plot(ax=axes, alpha=1, label='By day', color='tab:orange');
test_df[['timestamp', 'air_temperature']].set_index('timestamp').resample('D').mean()['air_temperature'].plot(ax=axes, alpha=1, color='tab:orange', label='');
axes.legend();
axes.text(train_df['timestamp'].iloc[9000000], -3, 'Train', fontsize=16);
axes.text(test_df['timestamp'].iloc[29400000], 30, 'Test', fontsize=16);
axes.axvspan(test_df['timestamp'].min(), test_df['timestamp'].max(), facecolor='green', alpha=0.2);

In [ ]:
# siteごとに確認する
fig, axes = plt.subplots(8, 2, figsize=(14, 30), dpi=100)
# site_idのidでループを回す
for i in range(train_df['site_id'].nunique()):
    train_df[train_df['site_id']==i][['timestamp', 'air_temperature']].set_index('timestamp').resample('H').mean()['air_temperature'].plot(ax=axes[i%8][i//8], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean temperature', fontsize=13);
    test_df[test_df['site_id']==i][['timestamp', 'air_temperature']].set_index('timestamp').resample('H').mean()['air_temperature'].plot(ax=axes[i%8][i//8], alpha=0.8, color='tab:blue', label='').set_xlabel('')
    train_df[train_df['site_id']==i][['timestamp', 'air_temperature']].set_index('timestamp').resample('D').mean()['air_temperature'].plot(ax=axes[i%8][i//8], alpha=1, label='By day', color='tab:orange')
    test_df[test_df['site_id']==i][['timestamp', 'air_temperature']].set_index('timestamp').resample('D').mean()['air_temperature'].plot(ax=axes[i%8][i//8], alpha=1, color='tab:orange', label='').set_xlabel('')
    axes[i%8][i//8].legend()
    axes[i%8][i//8].set_title('site_id {}'.format(i), fontsize=13);
    axes[i%8][i//8].axvspan(test_df['timestamp'].min(), test_df['timestamp'].max(), facecolor='green', alpha=0.2);
    plt.subplots_adjust(hspace=0.45)

In [ ]:
# 気象情報のみで補間してみます
# weather_train/test_dfは先ほど、削除してしまったので、改めて読み込みます
weather_train_df = pd.read_csv('../input/ashrae-energy-prediction/weather_train.csv')
weather_test_df = pd.read_csv('../input/ashrae-energy-prediction/weather_test.csv')

weather_train_df.head()

In [ ]:
print('train期間中のair_temperatureの欠損数: ', len(weather_train_df[weather_train_df.air_temperature.isnull()==True]))
print('test期間中のair_temperatureの欠損数: ', len(weather_test_df[weather_test_df.air_temperature.isnull()==True]))

In [ ]:
# 時間情報を利用しやすくするためのデータの型変換
weather_train_df['timestamp'] = pd.to_datetime(weather_train_df['timestamp'])
weather_test_df['timestamp'] = pd.to_datetime(weather_test_df['timestamp'])

# 気温は月・日・時の影響を受けると思われるので、特徴量を追加します
weather_train_df['hour'] = weather_train_df['timestamp'].dt.hour
weather_train_df['day'] = weather_train_df['timestamp'].dt.day
weather_train_df['month'] = weather_train_df['timestamp'].dt.month

weather_test_df['hour'] = weather_test_df['timestamp'].dt.hour
weather_test_df['day'] = weather_test_df['timestamp'].dt.day
weather_test_df['month'] = weather_test_df['timestamp'].dt.month

In [ ]:
# X:欠損がないデータ　X_dash:欠損があるデータ
X = weather_train_df[weather_train_df.air_temperature.isnull()==False]
X_dash = weather_train_df[weather_train_df.air_temperature.isnull()==True]

In [ ]:
# 予測（補間）する対象を目的変数として、yで定義する
y = X['air_temperature'].values

# 今回は、説明変数として['site_id', 'hour', 'day', 'month']だけを用います（場所と時間）
x = X[['site_id', 'hour', 'day', 'month']].values
x_dash = X_dash[['site_id', 'hour', 'day', 'month']].values

from sklearn.linear_model import LinearRegression
lr = LinearRegression() # インスタンスの作成

lr.fit(x, y) # 説明変数と真の目的変数を与えて、回帰モデルの学習s
predict_y = lr.predict(x_dash)

# X_dashの"air_temperature"カラムの値を予測値で置き換える
X_dash['air_temperature'] = predict_y

# dataframeを結合すれば、元のweather_train_dfと同じ形状で、'air_temperature'カラムが補間されたデータが作成できます
df = pd.concat([X, X_dash]) # ==(nearly equal) weather_train_df

del X, X_dash, predict_y
gc.collect()

# testデータでは、学習は行わず、予測だけ行います
X = weather_test_df[weather_test_df.air_temperature.isnull()==False] 
X_dash = weather_test_df[weather_test_df.air_temperature.isnull()==True] #欠損があるもの

x_dash = X_dash[['site_id', 'hour', 'day', 'month']].values

predict_y = lr.predict(x_dash)

X_dash['air_temperature'] = predict_y

df_dash = pd.concat([X, X_dash])

del X, X_dash, predict_y
gc.collect()

In [ ]:
# 月
train_df['month'] = train_df['timestamp'].dt.month
test_df['month'] = test_df['timestamp'].dt.month
# 日
train_df['day'] = train_df['timestamp'].dt.day
test_df['day'] = test_df['timestamp'].dt.day
# 時
train_df['hour'] = train_df['timestamp'].dt.hour
test_df['hour'] = test_df['timestamp'].dt.hour
# 曜日
train_df['weekday'] = train_df['timestamp'].dt.weekday
test_df['weekday'] = test_df['timestamp'].dt.weekday

In [ ]:
train_data = train_df['hour'].value_counts(dropna=False, normalize=True).sort_index().values
ind = np.arange(len(train_data))
width = 0.35

fig, axes = plt.subplots(1,1,figsize=(14, 6), dpi=100)
tr = axes.bar(ind, train_data, width, color='royalblue')

test_data = test_df['hour'].value_counts(dropna=False, normalize=True).sort_index().values
tt = axes.bar(ind+width, test_data, width, color='seagreen')

axes.set_ylabel('Normalized number of observations');
axes.set_xlabel('Hour');
axes.set_xticks(ind + width / 2)
axes.set_xticklabels(train_df['hour'].value_counts().sort_index().index, rotation=0)
axes2 = axes.twinx()
mr = axes2.plot(ind, train_df[['hour', 'meter_reading']].groupby('hour')['meter_reading'].mean().sort_index().values, 'D-', color='tab:orange', label='Mean meter reading');
axes2.grid(False);
axes2.tick_params(axis='y', labelcolor='tab:orange');
axes2.set_ylabel('Mean meter reading by hour', color='tab:orange');
axes.legend([tr, tt], ['Train', 'Test'], facecolor='white');
axes2.legend(loc=2, facecolor='white');

In [ ]:
# site_idと地域を結びつけるdictionary
country = {0:1,1:2,2:1,3:1,4:1,5:2,6:1,7:3,8:1,9:1,10:1,11:3,12:4,13:1,14:1,15:1} 
train_df['country'] = train_df['site_id'].map(country) 
# siteとGMT（グリニッジ標準時）との差（だと思います）
timediff = {0:4,1:0,2:7,3:4,4:7,5:0,6:4,7:4,8:4,9:5,10:7,11:4,12:0,13:5,14:4,15:4} 
train_df['time_diff']= train_df['site_id'].map(timediff)

# GMTとの差の情報を用いて'hour'カラムの値を変換
train_df['hour'] = train_df['hour'] + train_df['time_diff'] 
train_df.loc[train_df['hour']>23, 'hour'] = train_df[train_df['hour']>23]['hour'] - 24